In [7]:
import numpy as np
import matplotlib as plt
# import huggingface_hub as hub
# from huggingface_hub import hf_hub_download
%pip install datasets
from datasets import load_dataset
from sklearn.model_selection import train_test_split


Note: you may need to restart the kernel to use updated packages.


In [2]:
ds = load_dataset("artem9k/ai-text-detection-pile")

In [3]:
type(ds)

datasets.dataset_dict.DatasetDict

In [17]:
# Load the data into targets and texts then split into train and test sets

unsplit = ds['train']
# print(unsplit[83:86])
targets = unsplit['source']
ids = unsplit['id']
texts = unsplit['text']

# print(unsplit[1])

X_train, X_test, t_train, t_val  = train_test_split(texts, targets, test_size = 0.2, random_state=2013)

{'source': 'human', 'id': 1, 'text': '20+ Social Media Post Ideas to Radically Simplify Your Marketing Essay\n\nSocial Media Examiner’s (2021) video on social media initiatives was most useful to me since it provides more than twenty social media post ideas to improve the marketing of any business. This source demonstrates that social media, namely Facebook Business and Instagram, have great power and enhance marketing, promoting organizations through posts, stories, hashtags, and user-generated content. The most useful information to me was about trends, pros and cons, comparison, direct offer, and long-form content.\n\nEffective change leadership is important when an organization wants to adopt social media initiatives. A good leader is associated with effective communication, for example, written content on social media. For example, a leader may encourage customers to generate content by creating a hashtag, like #MyInstagramLogo, and inquiring them to share their views on the logo 

KeyError: (0, slice(None, 5, None))